In [1]:
import pandas as pd
import numpy as np
import re
import zipfile
import pylab as plt    
import seaborn as sns
import sys
from datetime import datetime, time
from jupyterthemes import jtplot
jtplot.style()

from functions import *

pd.set_option('display.max_columns', None)  # me muestre todas las columnas

In [56]:
shark=pd.read_csv('../data/sharkattack_parte3.csv')

Una vez que ya esta limpia la tabla de valores nulos e inconsistentes, el siguiente paso es analizarla para encontrar la mejor manera de poder obtener información relevante para el objetivo que me he impuesto.  
  
El objetivo es relacionar las cinco especies de tiburones con mayor cantidad de ataques, con la actividad que estaban realizando los atacados y con las fechas en las que se producen, para así poder establecer un periodo anual en el eque incrementar la seguridad mientras se realizan las diferentes actividades.
  
Para ello, las principales columnas que voy a necesitar son las de:
1. date: Ok
2. year: Ok
3. hora: Ok
4. activity: Ok
5. sex: Ok
6. fatal: Ok
7. species: Ok

In [57]:
shark.head()

,casenumber,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,species,investigator,pdf,href,href1,casenumber1,casenumber2,originalorder,unnamed,unnamed1,time,date1,hora,minutos
0,2018.06.25,25-Jun-2018,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,0,0,18:00:00,25-Jun-2018,18,0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,0,0,14:00:00,18-Jun-2018,14,0
2,2018.06.09,09-Jun-2018,2018,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,0,0,07:45:00,09-Jun-2018,7,45
3,2018.06.08,08-Jun-2018,2018,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,-1,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,0,0,00:01:00,08-Jun-2018,25,0
4,2018.06.04,04-Jun-2018,2018,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,-1,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,0,0,00:01:00,04-Jun-2018,25,0


---

# Date

In [58]:
shark.date[:10]

0    25-Jun-2018
1    18-Jun-2018
2    09-Jun-2018
3    08-Jun-2018
4    04-Jun-2018
5    03-Jun-2018
6    03-Jun-2018
7    27-May-2018
8    26-May-2018
9    26-May-2018
Name: date, dtype: object

In [59]:
meses_dict = {'Ene': 1,'Jan': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12,'Dec': 12,}
shark['mes'] = shark['date'].replace(meses_dict, regex=True)

In [60]:
shark.head()

,casenumber,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,species,investigator,pdf,href,href1,casenumber1,casenumber2,originalorder,unnamed,unnamed1,time,date1,hora,minutos,mes
0,2018.06.25,25-Jun-2018,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,0,0,18:00:00,25-Jun-2018,18,0,6
1,2018.06.18,18-Jun-2018,2018,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,0,0,14:00:00,18-Jun-2018,14,0,6
2,2018.06.09,09-Jun-2018,2018,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,0,0,07:45:00,09-Jun-2018,7,45,6
3,2018.06.08,08-Jun-2018,2018,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,-1,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,0,0,00:01:00,08-Jun-2018,25,0,6
4,2018.06.04,04-Jun-2018,2018,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,-1,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,0,0,00:01:00,04-Jun-2018,25,0,6


In [61]:
shark['mes'].apply(type).value_counts()

mes
<class 'int'>    5737
<class 'str'>     565
Name: count, dtype: int64

In [62]:
shark.loc[shark['mes'].apply(lambda x: isinstance(x, str)), 'mes'].unique()

array(['2018', '2017', '2015', '2014', '2013', '2012', '2011', '2010',
       '2009', '2008', '2006', '2004', '2001', '2000', '1999', '1998',
       '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989',
       '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981',
       '1980', '1979', '1978', '1976', '1975', '1974', '1973', '1972',
       '1971', '1970', '1969', '1968', '1966', '1965', '1964', '1963',
       '1962', '1961', '1960', '2176', '1959', '1958', '1957', '1956',
       '1955', '1954', '1953', '1952', '1951', '1950', '1949', '1948',
       '1946', '1945', '1944', '1943', '1942', '1941', '1940', '1939',
       '1938', '1937', '1936', '1932', '1931', '1930', '1929', '1928',
       '1927', '1926', '1925', '1923', '1922', '1920', '1919', '1918',
       '1917', '1915', '1914', '1913', '1912', '1911', '1910', '1909',
       '1907', '1906', '1905', '1904', '1903', '1901', '1900', '1899',
       '1898', '1897', '1896', '1895', '1893', '1892', '1890', '1888',
      

In [63]:
shark.date1.isna().sum()

7

In [64]:
shark.mes.value_counts()

mes
7       662
8       598
9       556
1       528
6       497
       ... 
1928      1
2013      1
1976      1
1865      1
1868      1
Name: count, Length: 199, dtype: int64

In [65]:
meses = [1,2,3,4,5,6,7,8,9,10,11,12]
for i, valor in enumerate(shark['mes']):
    if valor not in meses:
        shark.at[i, 'mes'] = 0

In [66]:
shark.mes.value_counts()

mes
7     662
8     598
0     565
9     556
1     528
6     497
4     451
10    444
12    436
3     412
11    407
2     380
5     366
Name: count, dtype: int64

---

Una vez que tengo los meses, el siguiente paso es establecer el año, para ello voy a revisar la columna de year

In [67]:
shark.year.unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997,
       1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989, 1969, 1988,
       1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976,
       1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966, 1965, 1964,
       1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953,
       1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945, 1944, 1943,
       1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933, 1932,
       1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922, 1921,
       1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911, 1910,
       1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900, 1899,
       1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889, 1888,
       1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878, 1877,
       1876, 1875, 1874, 1873, 1872, 1871, 1870, 18

In [68]:
(shark['year'] == 500).sum(), (shark['year'] == 77).sum(), (shark['year'] == 5).sum(), (shark['year'] == 0).sum()

(1, 1, 1, 125)

---

Una vez que tengo los ayos, el siguiente paso es establecer la hora, para ello voy a revisar la columna de hora

In [69]:
shark.hora.unique()

array([18, 14,  7, 25, 17,  9, 15,  8, 11, 10, 16, 21, 19, 12, 13, 23,  6,
        2, 22,  4,  5,  3, 20,  0,  1], dtype=int64)

Para que sea más facil de analizar lo voy a dividir en 4 franjas:
1. Mañana: 6 - 12
2. Tarde: 12 - 19
3. Noche: 19 - 0
4. Madrugada: 0 - 6

In [70]:
for valor in shark.hora:
    if 6 <= valor < 12: 
        shark['hora'].replace(valor, 'mañana', inplace=True)
    elif 12 <= valor < 19:
        shark['hora'].replace(valor, 'tarde', inplace=True)
    elif 19 <= valor <= 23:
        shark['hora'].replace(valor, 'noche', inplace=True)
    elif 0 <= valor < 6:
        shark['hora'].replace(valor, 'madrugada', inplace=True)
    else:
        shark['hora'].replace(valor, 'unknown', inplace=True)

In [71]:
shark.hora.unique()

array(['tarde', 'mañana', 'unknown', 'noche', 'madrugada'], dtype=object)

In [72]:
shark.head()

,casenumber,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,species,investigator,pdf,href,href1,casenumber1,casenumber2,originalorder,unnamed,unnamed1,time,date1,hora,minutos,mes
0,2018.06.25,25-Jun-2018,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,0,0,18:00:00,25-Jun-2018,tarde,0,6
1,2018.06.18,18-Jun-2018,2018,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,0,0,14:00:00,18-Jun-2018,tarde,0,6
2,2018.06.09,09-Jun-2018,2018,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,unknown shark,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,0,0,07:45:00,09-Jun-2018,mañana,45,6
3,2018.06.08,08-Jun-2018,2018,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,-1,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,0,0,00:01:00,08-Jun-2018,unknown,0,6
4,2018.06.04,04-Jun-2018,2018,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,-1,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,0,0,00:01:00,04-Jun-2018,unknown,0,6


---

# Activity

In [73]:
shark.activity = [e.lower() for e in shark.activity]

In [74]:
actividades = shark.activity.value_counts().to_dict()

In [77]:
shark.loc[shark['activity'].str.contains('surf|board', case=False, regex=True), 'activity'] = 'surfing'
shark.loc[shark['activity'].str.contains('div|snork', case=False, regex=True), 'activity'] = 'diving'
shark.loc[shark['activity'].str.contains('swim|bath|wading|float', case=False, regex=True), 'activity'] = 'swimming'
shark.loc[shark['activity'].str.contains('fish|stand|walk|tread', case=False, regex=True), 'activity'] = 'fishing'
shark.loc[shark['activity'].str.contains('boat|sail|kaya|kite|cano|row', case=False, regex=True), 'activity'] = 'sailing'
shark.loc[shark['activity'].str.contains('paddl', case=False, regex=True), 'activity'] = 'paddeling'

El resto de casos los considero demasiado particulares y los voy a catalogar como 'other'

In [78]:
act = ['surfing', 'diving', 'activity', 'fishing', 'sailing', 'paddeling']
shark['activity'] = shark['activity'].where(shark['activity'].isin(act), 'other')

In [79]:
(shark['activity'] == 'other').mean() * 100

41.09806410663281

Aunque es un % representativo, debido a la granularidad de los datos he decidido agruparlo así.

In [80]:
shark.activity.value_counts()

activity
other        2590
surfing      1562
fishing      1280
diving        674
sailing       184
paddeling      12
Name: count, dtype: int64

---

# Sex

In [81]:
shark.sex.value_counts()

sex
M    5096
F     637
u     569
Name: count, dtype: int64

La columna está ya limpia y se puede trabajar con ella

---

# Fatal

In [82]:
shark.fatal.value_counts()

fatal
N    4301
Y    1390
U     611
Name: count, dtype: int64

La columna está ya limpia y se puede trabajar con ella

---

# Species

In [83]:
shark.species.value_counts()

species
unknown shark                                                                                                                    2838
White shark                                                                                                                       163
Shark involvement prior to death was not confirmed                                                                                105
Invalid                                                                                                                           102
Shark involvement not confirmed                                                                                                    88
                                                                                                                                 ... 
1.2 m to 1.5 m [4.5' to 5'] shark                                                                                                   1
Bull shark, 2.3 m [7.5']                              

In [84]:
shark.species[:10]

0        White shark
1      unknown shark
2      unknown shark
3          2 m shark
4    Tiger shark, 3m
5      unknown shark
6        Tiger shark
7    Lemon shark, 3'
8     Bull shark, 6'
9      unknown shark
Name: species, dtype: object

In [85]:
shark.loc[shark['species'].str.contains('not c', case=False, regex=True), 'species'] = 'unknown shark'

Al igual que en la columna hora, para que sea más facil de analizar lo voy a agrupar en las 6 especiens más representativas:
  
Según una rápida investigación en internet las 6 especies que más atacan a los humanos son:
  
1. White Shark
2. Tiger shark
3. Spinner shark
4. Bull shark
5. Reef shark
6. Hammerhead shark
  
Así que las agruparé según esa lista y a los que no se puedan agrupar así, los calificaré de other shark

In [86]:
shark.species.value_counts()

species
unknown shark                                                                                                                    3064
White shark                                                                                                                       163
Invalid                                                                                                                           102
Tiger shark                                                                                                                        73
Shark involvement prior to death unconfirmed                                                                                       68
                                                                                                                                 ... 
Spinner shark, 1.2 m to 1.5 m [4' to 5']                                                                                            1
1 m shark, possibly whitetip reef shark               

In [87]:
shark.species = [e.lower() for e in shark.species]

In [88]:
shark.loc[shark['species'].str.contains('not c|invalid', case=False, regex=True), 'species'] = 'unknown shark'
shark.loc[shark['species'].str.contains('white', case=False, regex=True), 'species'] = 'white shark'
shark.loc[shark['species'].str.contains('tiger', case=False, regex=True), 'species'] = 'tiger shark'
shark.loc[shark['species'].str.contains('spinner', case=False, regex=True), 'species'] = 'spinner shark'
shark.loc[shark['species'].str.contains('bull', case=False, regex=True), 'species'] = 'bull shark'
shark.loc[shark['species'].str.contains('reef', case=False, regex=True), 'species'] = 'reef shark'
shark.loc[shark['species'].str.contains('hamme', case=False, regex=True), 'species'] = 'hammerhead shark'

In [89]:
act = ['white shark', 'tiger shark', 'spinner shark', 'bull shark', 'reef shark', 'hammerhead shark', 'unknown shark']
shark['species'] = shark['species'].where(shark['species'].isin(act), 'other shark')

In [90]:
shark.species.value_counts()

species
unknown shark       3167
other shark         1849
white shark          666
tiger shark          283
bull shark           177
reef shark            64
spinner shark         48
hammerhead shark      48
Name: count, dtype: int64

---

Llegados a este punto considero que ya tengo todos los datos necesarios para realizar el análisis, por lo que creo un nuevo df solo con las columnas que me interesan y lo exporto para analizarlo en la parte 5

In [37]:
shark.to_csv('../data/sharkattack_parte4.csv', index=False)

In [91]:
shark_analisis = pd.DataFrame(shark, columns=['mes', 'year', 'hora', 'activity', 'sex', 'fatal', 'species'])

In [42]:
shark_analisis.to_csv('../data/sharkattack_análisis.csv', index=False)

[SharkAttack - Parte 5 (Bonus: Análisis de hipótesis)](./SharkAttack-Parte5.ipynb)